In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from Script import Album, Artist, Music, Genre, Category, GenreCategory, MusicArtist, AlbumMusic, GenreArtist, Play, Favorite
from Script.SQLConn import SQLConn
from Script.RAMData import RAMData
from Script.SQLData import SQLData
import json
import csv
from difflib import SequenceMatcher
from tqdm.notebook import tqdm
from tkinter import Tk     # from tkinter import Tk for Python 3.x
from tkinter.filedialog import askopenfilename, askdirectory
from pathlib import Path
from os import listdir
from os.path import isfile, join

In [ ]:
## Postgres

# Connect to Database
print('Connecting to the Database...')
SQLConn.instance()

# Erase all tables
SQLConn.instance().cursor.execute("DROP TABLE IF EXISTS album,artist,music,genre,category,genre_category,music_artist,album_music,genre_artist,play,favorite CASCADE;")

# Create tables
sql_createtables_string = open('create_tables.sql', 'r', encoding="utf-8").read()
SQLConn.instance().cursor.execute(sql_createtables_string)

# RAM Started
ramData = RAMData.instance()

# SQL Started
sqlData = SQLData.instance()

# Commit and create Tables
SQLConn.instance().conn.commit()
print('Connected!')

In [ ]:
def formatName(name):
    return name.replace('\'', '\'\'')

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

# Artists

In [ ]:
f = open('../../Datasets/spotify_artists.json', 'r', encoding="utf8")
artists_file = json.load(f)
f.close()

In [ ]:
keywords_genres = {
    'Pop': ['pop', 'alt z'], 
    'Rock': ['rock'], 
    'Classical': ['classical', 'cello', 'orchestra', 'baroque'], 
    'Jazz': ['jazz', 'bossa nova', 'big band', 'hard bop', 'bebop'], 
    'Blues': ['blues'], 
    'Folk': ['folk'], 
    'Funk': ['funk'], 
    'Lo-fi': ['lo-fi'], 
    'Indie': ['indie', 'alt z'], 
    'Metal': ['metal'], 
    'Hip hop': ['hip hop'], 
    'Edm': ['electronica', 'edm', 'house', 'moog', 'electronic', 'techno', 'vaporwave', 'tronica', 'dubstep', 'trance'],
    'Rap': ['rap'],
    'Soundtrack': ['soundtrack'],
    'Sertanejo': ['sertanejo'],
    'R&b': ['r&b', 'soul'],
    'Punk': ['punk'],
    'Reggae': ['reggae'],
    'Country': ['country'],
    'Instrumental': ['instrumental', 'ambient'],
    'Nacional': ['brasileiro', 'brazilian', 'brasileiros', 'brasileira', 'nacional'],
}

In [ ]:
for artist_raw in tqdm(artists_file):
    artist = {
        'artist_spotify_uri': artist_raw['uri'],
        'name': formatName(artist_raw['name']),
        'followers': artist_raw['followers']['total'],
        'popularity': artist_raw['popularity']
    }
    artist_id = Artist.getArtistId(artist)
    
    for genre_raw in artist_raw['genres']:
        genre = {
            'genre': formatName(genre_raw)
        }
        genre_id = Genre.getGenreId(genre)
        genre_artist = {
            'genre_id': genre_id,
            'artist_id': artist_id
        }
        GenreArtist.saveGenreArtist(genre_artist)
        
        for category_name in keywords_genres:
            for key in keywords_genres[category_name]:
                if key in genre_raw:
                    genre_category = {
                        'genre_id': genre_id,
                        'category_id': Category.getCategoryId({'category': category_name})
                    }
                    GenreCategory.saveGenreCategory(genre_category)
    
sqlData.save()
SQLConn.instance().conn.commit()

# Musics

In [ ]:
f = open('../../Datasets/spotify_tracks.json', 'r', encoding="utf8")
musics_file = json.load(f)
f.close()

In [ ]:
for music_raw in tqdm(musics_file):
    music = {
        'music_spotify_uri': music_raw['uri'],
        'name': formatName(music_raw['name']),
        'duration_ms': music_raw['duration_ms'],
        'popularity': music_raw['popularity']
    }
    music_id = Music.getMusicId(music)
    for artist in music_raw['artists']:
        music_artist = {
            'music_id': music_id,
            'artist_id': Artist.getArtistId({'artist_spotify_uri': artist['uri']})
        }
        MusicArtist.saveMusicArtist(music_artist)

sqlData.save()
SQLConn.instance().conn.commit()

# Features

In [ ]:
f = open('../../Datasets/spotify_features.json', 'r', encoding="utf8")
features_file = json.load(f)
f.close()

In [ ]:
for feature_raw in tqdm(features_file):
    if feature_raw is None:
        continue
    music = {
        'music_id': Music.getMusicId({'music_spotify_uri': feature_raw['uri']}),
        'danceability': feature_raw['danceability'],
        'energy': feature_raw['energy'],
        'music_key': feature_raw['key'],
        'loudness': feature_raw['loudness'],
        'mode': feature_raw['mode'],
        'speechiness': feature_raw['speechiness'],
        'acousticness': feature_raw['acousticness'],
        'instrumentalness': feature_raw['instrumentalness'],
        'liveness': feature_raw['liveness'],
        'valence': feature_raw['valence'],
        'tempo': feature_raw['tempo'],
        'time_signature': feature_raw['time_signature'],
    }
    
    Music.updateMusic(music)
    
sqlData.save()
SQLConn.instance().conn.commit()

# Albums

In [ ]:
f = open('../../Datasets/spotify_albums.json', 'r', encoding="utf8")
albums_file = json.load(f)
f.close()

In [ ]:
for album_raw in tqdm(albums_file):
    label = album_raw.get('label', '')
    if label == None:
        label = ''
        
    release_date = ''
    if album_raw['release_date_precision'] == 'day':
        release_date = album_raw['release_date']
    elif album_raw['release_date_precision'] == 'month':
        release_date = album_raw['release_date'] + '-01'
    elif album_raw['release_date_precision'] == 'year':
        release_date = album_raw['release_date'] + '-01-01'
        
    album = {
        'album_spotify_uri': album_raw['uri'],
        'title': formatName(album_raw['name']),
        'release_date': release_date,
        'total_tracks': album_raw['total_tracks'],
        'label': formatName(label),
        'popularity': album_raw['popularity'],
    }
    album_id = Album.getAlbumId(album)
    for track in album_raw['tracks']['items']:
        album_music = {
            'album_id': album_id,
            'music_id': Music.getMusicId({'music_spotify_uri': track['uri']})
        }
        AlbumMusic.saveAlbumMusic(album_music)
        
sqlData.save()
SQLConn.instance().conn.commit()

# Play

In [ ]:
folder_selected = 'C:/Users/Matheus Ervilha/Google Drive/# PUC/13. TCC/Datasets/6. Spotify MyData Technical Log and Extended History'

onlyfiles = [f for f in listdir(folder_selected) if isfile(join(folder_selected, f))]
history_files = []
for file in onlyfiles:
    if 'endsong' in file:
        history_files.append(file)

for file in history_files:
    print('File: ' + file)
    f = open(folder_selected + '/' + file, encoding="utf8")
    data = json.load(f)
    for play_raw in tqdm(data):
        if 'spotify_track_uri' not in play_raw or play_raw['spotify_track_uri'] is None:
            continue
        play = {
            'music_id': Music.getMusicId({'music_spotify_uri': play_raw['spotify_track_uri']}),
            'end_time': play_raw['ts'],
            'ms_played': play_raw['ms_played'],
            'platform': play_raw['platform'],
            'ip_addr': play_raw['ip_addr_decrypted'],
            'reason_start': play_raw['reason_start'],
            'reason_end': play_raw['reason_end'],
            'shuffle': play_raw['shuffle'],
            'skipped': play_raw['skipped'],
            'offline': play_raw['offline'],
        }
        Play.savePlay(play)
        
sqlData.save()
SQLConn.instance().conn.commit()

# Tops

In [ ]:
folder_selected = 'C:/Users/Matheus Ervilha/Google Drive/# PUC/13. TCC/Datasets/6. Spotify MyData Technical Log and Extended History/Playlists'

onlyfiles = [f for f in listdir(folder_selected) if isfile(join(folder_selected, f))]
history_files = []
for file in onlyfiles:
    f = open(folder_selected + '/' + file, encoding="utf8")
    data = json.load(f)
    date = file.split('.')[0].split('_')[0]
    for track in data['items']:
        favorite = {
            'added_at': track['added_at'],
            'year': date,
            'music_id': Music.getMusicId({'music_spotify_uri': track['track']['uri']})
        }
        Favorite.saveFavorite(favorite)
        
sqlData.save()
SQLConn.instance().conn.commit()

In [ ]:
# Save everything else remaining
sqlData.save()

## Commit changes
SQLConn.instance().conn.commit()
print('Done!')